In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from astropy import units as u
from astropy.coordinates import SkyCoord
import SAGA

from SAGA import ObjectCuts as C

In [ ]:
SAGA_DIR = ""
SAGA_DB = ""

In [ ]:
# Load SAGA database
saga = SAGA.QuickStart(local_dir=SAGA_DIR)
 
saga.database["combined_base"].remote.path = SAGA_DB
saga.database["combined_base"].download(overwrite=True)

In [ ]:
# Load catalogs
base = saga.object_catalog.load_combined_base_catalog()
base = saga.host_catalog.construct_host_query("paper3").filter(base)

sats = C.is_sat.filter(base)
hosts = C.is_host.filter(base)

df = pd.DataFrame(dict(sats))

saga_sats = df
%store saga_sats

print(df.columns.tolist())

In [ ]:
# sns.set(style="darkgrid", palette="viridis")
custom_style = {
    "figure.facecolor": "212946",
    "axes.facecolor": "212946",
    "savefig.facecolor": "212946", 
    "grid.color": "2A3459",
    "text.color": "0.9",
    "axes.labelcolor": "0.9",
    "xtick.color": "0.9",
    "ytick.color": "0.9",
    "font.family": "Bitstream Vera Sans",
    "grid.linestyle": "-",
    "lines.solid_capstyle": "round"
}

sns.set_style("darkgrid", custom_style)

<h2>Ellipticity calculations</h2>

<h4>Get cartesian data points</h4>

In [ ]:
df['X'] = SkyCoord(ra=df.RA*u.degree, dec=df.DEC*u.degree, frame='icrs').transform_to(SkyCoord(ra=df.HOST_RA*u.degree, dec=df.HOST_DEC*u.degree, frame='icrs').skyoffset_frame()).cartesian.y
df['Y'] = SkyCoord(ra=df.RA*u.degree, dec=df.DEC*u.degree, frame='icrs').transform_to(SkyCoord(ra=df.HOST_RA*u.degree, dec=df.HOST_DEC*u.degree, frame='icrs').skyoffset_frame()).cartesian.z
df['HOST_X'] = SkyCoord(ra=df.HOST_RA*u.degree, dec=df.HOST_DEC*u.degree, frame='icrs').transform_to(SkyCoord(ra=df.HOST_RA*u.degree, dec=df.HOST_DEC*u.degree, frame='icrs').skyoffset_frame()).cartesian.y
df['HOST_Y'] = SkyCoord(ra=df.HOST_RA*u.degree, dec=df.HOST_DEC*u.degree, frame='icrs').transform_to(SkyCoord(ra=df.HOST_RA*u.degree, dec=df.HOST_DEC*u.degree, frame='icrs').skyoffset_frame()).cartesian.z
df['X_REFL'] = -1*df['X']
df['HOST_X_REFL'] = -1*df['HOST_X']

saga_sats = df
%store saga_sats


df[['OBJID', 'RA', 'DEC', 'X', 'Y', 'HOSTID', 'HOST_RA', 'HOST_DEC', 'HOST_X', 'HOST_Y']]

<h3>Ellipticity calculations</h3>

<h4>Filter data</h4>

In [ ]:
# data = df[df['Mr'] < -12.35]
data = df
data = data.groupby(['HOSTID']).filter(lambda x: len(x) > 2)
len(data)

In [ ]:
parent_ids = data['HOSTID'].unique()
len(parent_ids)
# ellipticity_df = pd.DataFrame({'HOSTID': parent_ids}, columns=['HOSTID', 'Q_xx', 'Q_yy', 'Q_xy'])

<h4>Quadrupole moments</h4>

In [ ]:
ellipticity_df = data.copy()
ellipticity_df['x_diff_sq'] = (ellipticity_df['X'] - ellipticity_df['HOST_X'])**2
ellipticity_df['y_diff_sq'] = (ellipticity_df['Y'] - ellipticity_df['HOST_Y'])**2
ellipticity_df['x_diff_y_diff'] = (ellipticity_df['Y'] - ellipticity_df['HOST_Y'])*(ellipticity_df['X'] - ellipticity_df['HOST_X'])
ellipticity_df = ellipticity_df.groupby('HOSTID', as_index = False)['x_diff_sq', 'y_diff_sq', 'x_diff_y_diff'].mean().rename(columns={'x_diff_sq':'Q_xx', 'y_diff_sq':'Q_yy', 'x_diff_y_diff':'Q_xy'})
ellipticity_df

<h4>Ellipticity components</h4>

In [ ]:
def e_1 (Q_xx, Q_yy, Q_xy):
    return (( Q_xx - Q_yy )/(Q_xx + Q_yy + 2*(Q_xx*Q_yy-Q_xy**2)**(1/2)))
def e_2 (Q_xx, Q_yy, Q_xy):
    return (2*Q_xy/(Q_xx + Q_yy + 2*(Q_xx*Q_yy-Q_xy**2)**(1/2)))
def e_3 (e_1, e_2):
    return ( e_1**2 + e_2**2 )**(1/2)


ellipticity_df['e_1'] = e_1(ellipticity_df['Q_xx'], ellipticity_df['Q_yy'], ellipticity_df['Q_xy'])
ellipticity_df['e_2'] = e_2(ellipticity_df['Q_xx'], ellipticity_df['Q_yy'], ellipticity_df['Q_xy'])
ellipticity_df['e_3'] = e_3(ellipticity_df['e_1'], ellipticity_df['e_2'])

ellipticity_df

In [ ]:
#play with binning
# sns.distplot(ellipticity_df["e_3"], bins=8, kde_kws=dict(gridsize=7, data=ellipticity_df, bw_method=None))
sns.histplot(ellipticity_df["e_3"], bins=7, element='poly', stat="density", fill=False)

In [ ]:
ellipticity_df_SAGA = ellipticity_df
%store ellipticity_df_SAGA

<h2>Highest luminosity analysis</h2>

In [ ]:
highest_luminosity_sats = df.groupby('HOSTID', as_index = False)['Mr'].min()

In [ ]:
highest_luminosity_ellipticity = pd.merge(highest_luminosity_sats, ellipticity_df, how="left", left_on="HOSTID", right_on="HOSTID")[['HOSTID', 'Mr', 'e_3']]

In [ ]:
highest_luminosity_ellipticity_SAGA = highest_luminosity_ellipticity[~highest_luminosity_ellipticity['e_3'].isna()]
%store highest_luminosity_ellipticity_SAGA
highest_luminosity_ellipticity_SAGA